<h3>Импорт библиотек</h3>
<p>
Импортируются все необходимые библиотеки для анализа данных, построения моделей, масштабирования признаков и вычисления метрик качества.
Также подключаются современные методы машинного обучения (Random Forest, XGBoost, CatBoost) и инструмент для перебора гиперпараметров <code>GridSearchCV</code>.
</p>
<hr>

In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix, classification_report
)
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

<h3>Загрузка данных</h3>
<p>
Здесь осуществляется загрузка датасета по оттоку клиентов — <b>Telco Customer Churn</b>.<br>
Этот датасет можно скачать на Kaggle по ссылке:<br>
<a href="https://www.kaggle.com/datasets/blastchar/telco-customer-churn" target="_blank">
https://www.kaggle.com/datasets/blastchar/telco-customer-churn
</a>
</p>
<hr>

In [12]:
customers_churn_df = pd.read_csv("../files/WA_Fn-UseC_-Telco-Customer-Churn.csv", sep=',')

<h3>Первичный просмотр данных</h3>
<p>
Отображение первых пяти строк датасета для быстрого знакомства с его структурой и содержимым.
</p>
<hr>

In [13]:
customers_churn_df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


<h3>Анализ размера датасета</h3>
<p>
Проверка количества строк и столбцов (размерности датасета) — важно для оценки объёма и структуры данных.
</p>
<hr>

In [14]:
customers_churn_df.shape

(7043, 21)

<h3>Информация о столбцах</h3>
<p>
Вывод структуры датафрейма: названия столбцов, типы данных, количество ненулевых значений, что важно для дальнейшей обработки и выявления пропусков.
</p>
<hr>

In [15]:
customers_churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


<h3>Формирование целевой переменной (target) и удаление лишних признаков</h3>
<p>
Столбец <code>Churn</code> переводится из текстового формата в числовой (0 — не ушёл, 1 — ушёл), чтобы использовать его как целевую переменную. <br>
Удаляются столбцы <code>customerID</code> и <code>Churn</code> из датафрейма признаков, так как они не нужны для обучения.
</p>
<hr>

In [16]:
y = customers_churn_df["Churn"].map({"No": 0, "Yes": 1})
customers_churn_df = customers_churn_df.drop(["customerID", "Churn"], axis=1)

<h3>Определение признаков для one-hot encoding</h3>
<p>
Создаётся список категориальных признаков, которые подлежат кодированию, и исключаются числовые признаки. <br>
Проводится one-hot encoding по выбранным столбцам с помощью <code>pd.get_dummies</code> — каждая уникальная категория превращается в отдельную бинарную колонку.
</p>
<hr>

In [17]:
exclude = ["SeniorCitizen", "tenure", "MonthlyCharges", "TotalCharges"]
included = [col for col in customers_churn_df.columns if col not in exclude]
customers_churn_df = pd.get_dummies(customers_churn_df, columns=included)

<h3>Проверка размеров после one-hot encoding</h3>
<p>
Анализируется размер датафрейма после кодирования категориальных переменных: обычно число столбцов существенно возрастает.
</p>
<hr>

In [18]:
customers_churn_df.shape

(7043, 45)

<h3>Просмотр обработанного датафрейма</h3>
<p>
Отображается одна строка датафрейма после всех преобразований — удобно для контроля итогового вида данных.
</p>
<hr>

In [19]:
customers_churn_df.head(1)

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,29.85,29.85,True,False,False,True,True,False,...,False,True,False,False,False,True,False,False,True,False


<h3>Обработка числовых признаков и пропусков</h3>
<p>
Числовые признаки <code>TotalCharges</code> и <code>MonthlyCharges</code> преобразуются к типу <code>float</code>. <br>
Пропущенные значения <code>TotalCharges</code> заполняются средним по колонке. <br>
Затем оба признака масштабируются в диапазон [0,1] с помощью <code>MinMaxScaler</code>.
</p>
<hr>

In [20]:
customers_churn_df['TotalCharges'] = pd.to_numeric(customers_churn_df['TotalCharges'], errors='coerce')
customers_churn_df['MonthlyCharges'] = pd.to_numeric(customers_churn_df['MonthlyCharges'], errors='coerce')
num_columns = ["MonthlyCharges", "TotalCharges"]
print("Before:")
print(customers_churn_df[num_columns].isna().sum())
print("-" * 100)
customers_churn_df["TotalCharges"] = customers_churn_df["TotalCharges"].fillna(customers_churn_df["TotalCharges"].mean())
print("After:")
print(customers_churn_df[num_columns].isna().sum())


scaler = MinMaxScaler()
customers_churn_df[num_columns] = scaler.fit_transform(customers_churn_df[num_columns])


Before:
MonthlyCharges     0
TotalCharges      11
dtype: int64
----------------------------------------------------------------------------------------------------
After:
MonthlyCharges    0
TotalCharges      0
dtype: int64


<h3>Финальный просмотр признаков</h3>
<p>
Повторная проверка первых строк и признаков после всех обработок — последний контроль перед разделением на выборки и построением моделей.
</p>
<hr>

In [21]:
customers_churn_df.head(1)

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,0.115423,0.001275,True,False,False,True,True,False,...,False,True,False,False,False,True,False,False,True,False


<h3>Формирование матрицы признаков</h3>
<p>
Определение X (матрицы признаков) для дальнейшего обучения моделей.
</p>
<hr>

In [22]:
X = customers_churn_df

<h3>Разделение на обучающую и тестовую выборки</h3>
<p>
Данные разбиваются на обучающую (train) и тестовую (test) выборки для честной оценки качества моделей.<br>
<code>test_size=0.2</code> означает, что 20% данных идут в тест, 80% — в обучение.
</p>
<hr>

In [23]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

<h3>Обучение и сравнение моделей</h3>
<p>
Последовательно обучаются и тестируются сразу несколько моделей:<br>
— <b>Logistic Regression</b><br>
— <b>Random Forest</b><br>
— <b>XGBoost</b><br>
— <b>CatBoost</b><br>
Для каждой рассчитываются метрики качества и результаты собираются в общую таблицу для сравнения.
</p>
<hr>

In [24]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    'CatBoost': CatBoostClassifier(verbose=0, random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-score": f1_score(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_proba)
    })

results_df = pd.DataFrame(results).set_index("Model")
print(results_df)

/Users/germangolod/JetbrainsProjects/PycharmProProjects/ML_Projects/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [13:41:07] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


                     Accuracy  Precision    Recall  F1-score   ROC-AUC
Model                                                                 
Logistic Regression  0.750177   0.517647  0.825737  0.636364  0.861968
Random Forest        0.792761   0.650558  0.469169  0.545171  0.834991
XGBoost              0.788502   0.620579  0.517426  0.564327  0.835457
CatBoost             0.801278   0.656566  0.522788  0.582090  0.856632


<h3>Поиск оптимальных параметров для CatBoost (GridSearchCV)</h3>
<p>
С помощью <code>GridSearchCV</code> проводится автоматизированный перебор гиперпараметров для CatBoost, чтобы найти лучшие параметры по метрике <b>ROC-AUC</b>.<br>
Это помогает улучшить итоговое качество модели.
</p>
<hr>

In [25]:
params = {
    "iterations": [100, 200],
    "depth": [4, 6, 8],
    "learning_rate": [0.01, 0.05, 0.1],
    "l2_leaf_reg": [1, 3, 5],
}
cat = CatBoostClassifier(verbose=0, random_state=42)

grid = GridSearchCV(cat, params, scoring="roc_auc", cv=3)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best ROC-AUC:", grid.best_score_)

Best params: {'depth': 4, 'iterations': 100, 'l2_leaf_reg': 5, 'learning_rate': 0.05}
Best ROC-AUC: 0.8464599069445539


<h3>Обучение CatBoost с лучшими параметрами</h3>
<p>
Модель CatBoost обучается с параметрами, найденными на предыдущем шаге, что позволяет получить максимальное качество.
</p>
<hr>

In [26]:
model = best_model = CatBoostClassifier(
    depth=4,
    iterations=100,
    l2_leaf_reg=5,
    learning_rate=0.05,
    verbose=0,
    random_state=42
)

In [27]:
model.fit(X_train, y_train)

<h3>Оценка финальной модели и вывод отчёта</h3>
<p>
Для финальной модели CatBoost рассчитываются все основные метрики (accuracy, precision, recall, F1-score, ROC-AUC) и строится confusion matrix.<br>
Также выводится полный classification report для анализа ошибок и сильных сторон модели.
</p>
<hr>

In [28]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

In [29]:
print("Accuracy:", f"{accuracy_score(y_test, y_pred):.4f}")
print("Precision:", f"{precision_score(y_test, y_pred):.4f}")
print("Recall:", f"{recall_score(y_test, y_pred):.4f}")
print("F1-score:", f"{f1_score(y_test, y_pred):.4f}")
print("ROC-AUC:", f"{roc_auc_score(y_test, y_proba):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nDetailed Report:\n", classification_report(y_test, y_pred, digits=4))

Accuracy: 0.8098
Precision: 0.6804
Recall: 0.5308
F1-score: 0.5964
ROC-AUC: 0.8644
Confusion Matrix:
 [[943  93]
 [175 198]]

Detailed Report:
               precision    recall  f1-score   support

           0     0.8435    0.9102    0.8756      1036
           1     0.6804    0.5308    0.5964       373

    accuracy                         0.8098      1409
   macro avg     0.7619    0.7205    0.7360      1409
weighted avg     0.8003    0.8098    0.8017      1409

